In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## INPUT ##

ms_cmd = "./ms 20 100 -t 120068.91955982734 -r 27899.061854409614 10000 -p 4 -eN 1.7346370798e-07 1.0 -eN 8.01653753135e-07 0.0008962320493712909 -eN 1.9631724918e-06 0.0004067588423180988 -eN 3.6460892761e-06 0.0013344148301248928 -eN 6.08446384525e-06 0.0036398062219619477 -eN 9.6173931125e-06 0.005205685373102609 -eN 1.1940308992999999e-05 0.0056877347538072795 -eN 1.47362865135e-05 0.005685812690513007 -eN 1.8101936854e-05 0.005357255424120209 -eN 2.2153110145e-05 0.004869386514186823 -eN 2.702947617e-05 0.004343204153097223 -eN 3.28991883535e-05 0.003844942881346934 -eN 3.99646304605e-05 0.0034036175351557234 -eN 4.84692460075e-05 0.0030266705914845987 -eN 5.87063665255e-05 0.0027103974694213614 -eN 7.1028706107e-05 0.0024471479739199675 -eN 8.5861520515e-05 0.002228520665568713 -eN 0.000103714600295 0.0020467366315367816 -eN 0.000125205590715 0.0018950531185080853 -eN 0.000151074067015 0.0017678830319484272 -eN 0.00018221201690000002 0.0016607112591100787 -eN 0.00021969299046499998 0.0015698842644952123 -eN 0.000264808745815 0.0014924914431123776 -eN 0.000319114223235 0.001426207050080056 -eN 0.00038448168076500003 0.0013691404805155308 -eN 0.00046316565689 0.0013198237815652064 -eN 0.0005578770946350001 0.0012770715775269872 -eN 0.000671881618455 0.0012399857071605637 -eN 0.00080910863824 0.0012078940116894608 -eN 0.0009742904361 0.0011802664210302557 -eN 0.00117311791025 0.001147839329644356 -eN 0.00170053166755 0.0011148285970524851 -eN 0.00246469278715 0.0010912241395123357 -eN 0.0035718985527 0.0010738548818618444 -eN 0.0051761188680499995 0.0010660612227918259"
mu = 3e-8

In [ ]:
cmd_ls = ms_cmd.split()

In [ ]:
## PARSE PARAMETERS ##
size_loc = int(cmd_ls[7])
print("Size of locus:", size_loc, "bp")

In [ ]:
N_0 = round(float(cmd_ls[4])/(4*mu*size_loc))
print("Current N_e:", N_0)

In [ ]:
#rho = float(cmd_ls[6])/(4*N_0*size_loc)
rho = mu*float(cmd_ls[6])/float(cmd_ls[4])
print("Recomb rate:", rho)

In [ ]:
## PAST-WARD DEMOGRAPHY ##
gen_pw = []
pop_size_pw = []

for i in range(cmd_ls.index("-eN"), len(cmd_ls), 3):
    gen_pw.append(int(float(cmd_ls[i+1])*4*N_0))
    pop_size_pw.append(int(float(cmd_ls[i+2])*N_0))
    
gen_pw = np.array(gen_pw)
pop_size_pw = np.array(pop_size_pw)

In [ ]:
#for i in range(len(gen_pw)): print(gen_pw[i], pop_size_pw[i]) 

In [ ]:
## CONVERSION TO FORWARD DEMOGRAPHY ##
max_gen_bp = 1e5

In [ ]:
mask = gen_pw < max_gen_bp

In [ ]:
gen = gen_pw[mask]
pop = pop_size_pw[mask]

In [ ]:
gen = np.flip(max_gen_bp - gen)
gen = np.concatenate(([0], gen[:-1], [max_gen_bp]))

In [ ]:
pop = np.concatenate((np.flip(pop), [pop[0]]))

In [ ]:
#for i in range(len(gen)): print(gen[i], pop[i]) 

In [ ]:
## SCALING ##
scaling_factor = 40

In [ ]:
scaled_mu = mu*scaling_factor
scaled_rho = rho*scaling_factor
scaled_gen = (gen/scaling_factor).astype(int)
scaled_pop = (pop/scaling_factor).astype(int)

In [ ]:
## Visualization ##
plt.figure(figsize=(20,10))
plt.step(gen_pw, pop_size_pw, where='post', lw=5, alpha=0.7)
plt.step(np.flip(max_gen_bp-gen), np.flip(pop), where='pre', lw=5, color='g', alpha=0.7)
#plt.step(np.flip(max_gen_bp/scaling_factor-scaled_gen), np.flip(scaled_pop), where='pre', lw=5, color='y', alpha=0.7)
plt.yscale("log")
plt.xscale("log")

In [ ]:
## save scaled demography to .param file ##
slim_param_f = "maize"
prec = 2
with open(slim_param_f+"_scale_"+str(scaling_factor)+".param", "w") as outF:
    print(np.format_float_scientific(scaled_mu, precision=prec), file=outF)
    print(np.format_float_scientific(scaled_rho, precision=prec), file=outF)
    for i in range(len(scaled_gen)):
        print(scaled_gen[i], scaled_pop[i], file=outF)

__Format of `.param` file__

```
<mut_rate>
<recomb_rate>
<gen_1> <Ne_1>
<gen_2> <Ne_2>
<gen_3> <Ne_3>
...
```